- Se instalan los módulos necesarios.

*Para instalar el scikit-surprice hace falta descargar la herramienta de desarrollo para el escritorio con C++ de visual studio community 2022, pues ya es un módulo obsoleto en la versión más actualizada de python.  https://visualstudio.microsoft.com/es/vs/community/*

In [ ]:
%pip install scikit-surprise
%pip install gradio

- Se importan las librerías.

In [1]:
import pandas as pd
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt
%matplotlib inline
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split, cross_validate

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- Se cargan los datasets:

In [2]:
df_score = pd.read_csv('EDAcsvResults\\ratings.csv')
df_score.head(2)


,userId,rating,movieId
0,1,1.0,as680
1,1,4.5,ns2186


In [3]:
df_platforms = pd.read_csv('EDAcsvResults\\platforms.csv')
df_platforms.head(2)

,movieId,type,title,rating,platform,score
0,as1,movie,the grand seduction,g,amazon prime,3.463048
1,as2,movie,take care good night,13+,amazon prime,3.566434


In [4]:
df_platforms = df_platforms[['movieId', 'title']]
df_platforms['title'] = df_platforms['title'].replace(' (Series)', '')
df_platforms = df_platforms.reset_index(drop=True)
df_platforms.head(1)

,movieId,title
0,as1,the grand seduction


- Se limita el dataset a 100000 filas.

- Se define la escala de calificación utilizada en el dataset.


- Se cargan los datos de rating en un formato compatible con la librería.

- Se dividen los datos en 'entrenamiento' y 'evaluación', en una relación 75-25 respectivamente.

In [5]:
rows = 200000 

reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df_score[['userId', 'movieId', 'rating']][:rows], reader)

trainset, testset = train_test_split(data, test_size=.25)

- Se entrena el modelo.

In [6]:
model = SVD()
model.fit(trainset)

- Se realizan las predicciones.

In [7]:
predicts = model.test(testset)
predicts[1]

Prediction(uid=1932, iid='as9306', r_ui=4.0, est=2.384531352891614, details={'was_impossible': False})

- Se realiza una predicción al azar para un usuario y película.

In [8]:
model.predict(1992, 'ds540')

Prediction(uid=1992, iid='ds540', r_ui=None, est=3.662015411991693, details={'was_impossible': False})

- Se selecciona un usuario al azar para hacer una recomendación completa.

- Se definen las películas con una calificación superior a 4 otorgada por el usuario.

- Se reinicia el índice de los datos filtrados.

- Se unen los datos filtrados con los datos de 'df_platforms'

In [9]:
user = 25192

min_score = 4
df_user = df_score[(df_score.userId == user) & (df_score.rating >= min_score)]
df_user = df_user.reset_index(drop=True)

df_user = pd.merge(df_user, df_platforms, on='movieId', how='left')
df_user.head(2)

,userId,rating,movieId,title
0,25192,5.0,as6837,vampire strippers
1,25192,5.0,as5281,hum tum


In [10]:
user_recomendation = df_platforms.iloc[:3691].copy()
print(user_recomendation.shape)
user_recomendation.head(2)

(3691, 2)


,movieId,title
0,as1,the grand seduction
1,as2,take care good night


- Se descartan las películas que ya vio el usuario.

In [11]:
watched = df_score[df_score['userId'] == user]
print(watched.shape)
watched.head(2)

(7, 3)


,userId,rating,movieId
2429875,25192,3.5,ns489
2429876,25192,5.0,as6837


- Se lleva a cabo la recomendación.

In [12]:
user_recomendation['estimate_rating'] = user_recomendation['movieId'].apply(lambda x: model.predict(user, x).est)

In [13]:
user_recomendation = user_recomendation.sort_values('estimate_rating', ascending=False)
user_recomendation.head(2)

,movieId,title,estimate_rating
480,as481,the primms,4.023874
1277,as1278,posse,4.002291


**Predicción con un usuario y película.**

- Se toma la predicción de la calificación del usuario.

- Si la predicción es de 3.5 o superior es una película recomendada, de lo contrario no lo es.

In [14]:
def movie_recomendation(userId, movieId):
    prediction = model.predict(userId, movieId)
    if prediction.est >=3.5:
        return 'Recomendada', prediction.est
    else:
        return 'No se recomienda', prediction.est
movie_recomendation(666, 'hs666')

('Recomendada', 3.5797446409863314)

- Se hace una evaluación del modelo.

In [15]:
accuracy.rmse(predicts)

RMSE: 0.9676


0.9675590896937462

- Se optimizan los hiperparámetros.

In [16]:
rmse_test = []
factors = [1, 2, 4, 8, 16, 32, 64, 128]

for factor in factors:
    print(factor)
    model = SVD(n_factors=factor)
    cv = cross_validate(model, data, measures=['RMSE'], cv=3, verbose=True)
    rmse_test.append(np.mean(cv['test_rmse']))


1
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9575  0.9579  0.9557  0.9570  0.0010  
Fit time          1.56    1.53    1.20    1.43    0.16    
Test time         1.42    0.99    0.78    1.06    0.26    
2
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9526  0.9615  0.9539  0.9560  0.0039  
Fit time          1.11    1.13    1.13    1.12    0.01    
Test time         0.95    0.92    0.92    0.93    0.01    
4
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9566  0.9526  0.9594  0.9562  0.0028  
Fit time          1.15    1.20    1.19    1.18    0.02    
Test time         0.77    0.92    0.76    0.82    0.07    
8
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9552  0.9

**Creación de la interfaz para el modelo.**

- Se crea una función que retorne el título de la película.

In [17]:
def title(movieId):
    return df_platforms[df_platforms.movieId == movieId].title.iloc[0].title()
title('hs369')

'The Waiting Room'

- Se crea una función que utilice la función de recomendación y la de título para retornar la recomendación, el puntaje y el título de la película.

In [18]:
def title_recomendation(userId, movieId):
    recomended, rating = movie_recomendation(userId, movieId)
    title_ = df_platforms[df_platforms.movieId == movieId].title.iloc[0].title()
    return (recomended, rating, title_)
title_recomendation(141213, 'ds1312')


('Recomendada', 3.709309891639033, 'The Muppets (Series)')

- Se crea una interfaz de usuario en Gradio para recomendar una película al usuario.

In [19]:
title = str("Gato recomendations")

with gr.Blocks(title=title) as demo:
    user = gr.inputs.Number(label='Tu número de usuario')
    movie = gr.Textbox(label='Id de película')
    recomendation = gr.Button('Busca tu pelicula')
    title = gr.Textbox(label='Tu película se llama')
    output = gr.Textbox(label='Quiere que la veas?')
    score = gr.Textbox(label='Qué tanto vale la pena')
    recomendation.click(fn = title_recomendation, inputs=[user, movie], outputs=[output, score, title])
demo.launch(share = True)

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\inputs.py:59: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://7bf835bb08c477d14f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
